In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import joblib
import warnings
import os 

In [ ]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_column', None) 

In [ ]:

print("--- 2. Carregando e Inspecionando Dados ---")
file_path = 	'furtos_agregado_por_cidades2.csv'

try:
    # Ler o arquivo Excel, usando a segunda linha como cabeçalho
    df = pd.read_csv(file_path, delimiter=';')
    print(f"Dados carregados com sucesso de {file_path}")
    print(f"Shape inicial: {df.shape}")
except FileNotFoundError:
    print(f"ERRO: Arquivo não encontrado em {file_path}")
    # Parar execução se o arquivo não for encontrado
    raise
except Exception as e:
    print(f"Erro ao carregar o arquivo: {e}")
    raise

In [ ]:
print("\nPrimeiras 5 linhas:")
print(df.head())

print("\nInformações das colunas e tipos de dados:")
df.info()

print("\nVerificação de valores ausentes:")
print(df.isnull().sum())

df['Area'] = df['Area'].str.replace(',','.',regex=True).astype(float)


In [ ]:

print("\n--- 3. Pré-processamento e Limpeza dos Dados ---")
# Limpar nomes das colunas (remover espaços extras)
df.columns = df.columns.str.strip()
print("Nomes das colunas após limpeza:", df.columns.tolist())

# Renomear coluna de Cidade
if 	'Rótulos de Linha	' in df.columns:
    df.rename(columns={	'Rótulos de Linha': 	'Cidade'}, inplace=True)
    print("Coluna 	'Rótulos de Linha	' renomeada para 	'Cidade	'")

# Definir colunas de interesse
target_col = 	'Total'
feature_cols = [	'População', 	'Area', 	'Densidade_Recalculada']
required_cols = [	'Cidade', target_col] + feature_cols

# Verificar se todas as colunas necessárias existem
if not all(col in df.columns for col in required_cols):
    missing = [col for col in required_cols if col not in df.columns]
    raise ValueError(f"ERRO: Colunas necessárias ausentes: {missing}")

# Selecionar apenas as colunas relevantes
df_processed = df[required_cols].copy()
print(f"\nShape após selecionar colunas: {df_processed.shape}")


In [ ]:
print("Garantindo tipos numéricos para features e target...")
for col in [target_col] + feature_cols:
    df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
    if df_processed[col].isnull().any():
        print(f"  AVISO: Valores não numéricos encontrados e convertidos para NaN em 	'{col}	'. Removendo linhas com NaN.")
        df_processed.dropna(subset=[col], inplace=True)

print("\nTipos de dados finais:")
df_processed.info()

print("\nDados pré-processados (primeiras 5 linhas):")
# display(df_processed.head()) # Replaced display with print
print(df_processed.head())

In [ ]:

print("\n--- 4. Divisão dos Dados (Treino, Validação, Teste) ---")
# Separar features (X) e target (y)
X = df_processed[feature_cols]
y = df_processed[target_col]

# Primeira divisão: Treino+Validação (80%) e Teste (20%)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Segunda divisão: Treino (75% de 80% = 60%) e Validação (25% de 80% = 20%)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

print(f"Tamanho do conjunto de Treino: {X_train.shape[0]} amostras")
print(f"Tamanho do conjunto de Validação: {X_val.shape[0]} amostras")
print(f"Tamanho do conjunto de Teste: {X_test.shape[0]} amostras")


In [ ]:

print("\n--- 5. Treinamento do Modelo de Regressão ---")
# Inicializar e treinar o modelo
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
print("Treinando o modelo RandomForestRegressor...")
model.fit(X_train, y_train)
print("Treinamento concluído.")

# Opcional: Salvar o modelo treinado
model_save_path = 	'regression_model_script.joblib	'
try:
    joblib.dump(model, model_save_path)
    print(f"Modelo treinado salvo em: {model_save_path}")
except Exception as e:
    print(f"Erro ao salvar o modelo: {e}")

In [ ]:

print("\n--- 6. Avaliação do Modelo ---")
# Fazer previsões
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

# Calcular métricas para o conjunto de validação
val_r2 = r2_score(y_val, y_val_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)
val_mse = mean_squared_error(y_val, y_val_pred)

# Calcular métricas para o conjunto de teste
test_r2 = r2_score(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

print("--- Métricas de Performance ---")
print("Conjunto de Validação:")
print(f"  R²: {val_r2:.4f}")
print(f"  MAE: {val_mae:.4f}")
print(f"  MSE: {val_mse:.4f}")
print("-----------------------------")
print("Conjunto de Teste:")
print(f"  R²: {test_r2:.4f}")
print(f"  MAE: {test_mae:.4f}")
print(f"  MSE: {test_mse:.4f}")

In [ ]:

print("\n--- 7. Visualização da Performance ---")
# Definir caminhos para salvar os gráficos
scatter_plot_path = 	'eda/regression/test_performance_scatter_script.png'
feature_plot_path = 	'eda/regression/feature_importance_script.png'

# Gráfico de Dispersão: Real vs. Previsto (Teste)
print(f"Gerando gráfico de dispersão em: {scatter_plot_path}")
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_test_pred, alpha=0.6, label=	'Previsões')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 	'--r', linewidth=2, label=	'Ideal (y=x)')
plt.xlabel("Valores Reais (Subtrações)")
plt.ylabel("Valores Previstos (Subtrações)")
plt.title("Performance: Real vs. Previsto (Conjunto de Teste)")
plt.legend()
plt.grid(True)
# plt.show() # Replaced show with savefig
try:
    plt.savefig(scatter_plot_path)
    print(f"Gráfico de dispersão salvo em {scatter_plot_path}")
except Exception as e:
    print(f"Erro ao salvar gráfico de dispersão: {e}")
plt.close() # Fechar a figura

# Gráfico de Importância das Features
print(f"Gerando gráfico de importância das features em: {feature_plot_path}")
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names = X_train.columns # Usar nomes das colunas de X_train

plt.figure(figsize=(10, 6))
plt.title("Importância das Features")
plt.bar(range(X_train.shape[1]), importances[indices], align="center")
plt.xticks(range(X_train.shape[1]), [feature_names[i] for i in indices], rotation=45, ha="right")
plt.ylabel("Importância Relativa")
plt.tight_layout()
# plt.show() # Replaced show with savefig
try:
    plt.savefig(feature_plot_path)
    print(f"Gráfico de importância salvo em {feature_plot_path}")
except Exception as e:
    print(f"Erro ao salvar gráfico de importância: {e}")
plt.close() # Fechar a figura